# Varational Autoencoders in ZhuSuan

### This code implements a VAE with Gaussian prior and Bernoulli likelihood based on ZhuSuan.
#### The framework has been setup, please only fill the space with "TODO" comments.
#### You may see some detailed instructions in the comments. For detailed usage of ZhuSuan, please see the documentation on http://zhusuan.readthedocs.io/en/latest/concepts.html

#### If you have any questions, please contact with me: lucheng.lc15@gmail.com

In [101]:
import os
import time
import tensorflow as tf
import numpy as np
import zhusuan as zs

import matplotlib.pyplot as plt
from skimage import io, img_as_ubyte
from skimage.exposure import rescale_intensity

## I. Load MNIST Dataset
We use tensorflow tools to load dataset for convenience.

In [102]:
(data_train, label_train), _ = tf.keras.datasets.mnist.load_data()

In [103]:
print(data_train.shape, label_train.shape)

(60000, 28, 28) (60000,)


## II. Some Utils for Training and Visualization

Here are some functions for next steps. You don't need to know the details.

In [104]:
def iterbatches(arrays, *, num_batches=None, batch_size=None, shuffle=True, include_final_partial_batch=True):
    assert (num_batches is None) != (batch_size is None), 'Provide num_batches or batch_size, but not both'
    n = arrays[0].shape[0]
    assert all(a.shape[0] == n for a in arrays[1:])
    inds = np.arange(n)
    if shuffle: np.random.shuffle(inds)
    sections = np.arange(0, n, batch_size)[1:] if num_batches is None else num_batches
    for batch_inds in np.array_split(inds, sections):
        if include_final_partial_batch or len(batch_inds) == batch_size:
            yield tuple(a[batch_inds] for a in arrays)

In [105]:
def save_image_collections(x, filename, shape=(10, 10), scale_each=False,
                           transpose=False):
    """
    :param shape: tuple
        The shape of final big images.
    :param x: numpy array
        Input image collections. (number_of_images, rows, columns, channels) or
        (number_of_images, channels, rows, columns)
    :param scale_each: bool
        If true, rescale intensity for each image.
    :param transpose: bool
        If true, transpose x to (number_of_images, rows, columns, channels),
        i.e., put channels behind.
    :return: `uint8` numpy array
        The output image.
    """
    n = x.shape[0]
    if transpose:
        x = x.transpose(0, 2, 3, 1)
    if scale_each is True:
        for i in range(n):
            x[i] = rescale_intensity(x[i], out_range=(0, 1))
    n_channels = x.shape[3]
    x = img_as_ubyte(x)
    r, c = shape
    if r * c < n:
        print('Shape too small to contain all images')
    h, w = x.shape[1:3]
    ret = np.zeros((h * r, w * c, n_channels), dtype='uint8')
    for i in range(r):
        for j in range(c):
            if i * c + j < n:
                ret[i * h:(i + 1) * h, j * w:(j + 1) * w, :] = x[i * c + j]
    ret = ret.squeeze()
    io.imsave(filename, ret)

## III. Build the Generative Model by ZhuSuan

Define the generative model according to the generative process.

* TODO: complete the network.
    

In [106]:
@zs.meta_bayesian_net(scope="gen", reuse_variables=True)
def build_gen(x_dim, z_dim, n, n_particles=1):
    bn = zs.BayesianNet()
    z_mean = tf.zeros([n, z_dim])
    z = bn.normal("z", z_mean, std=1., group_ndims=1, n_samples=n_particles)
    h = tf.layers.dense(z, 500, activation=tf.nn.relu)
    ''' 
        TODO: add one more mlp layer of 500 hidden units with h as the input and return h
    '''
    h = tf.layers.dense(h, 500, activation=tf.nn.relu)
    
    x_logits = tf.layers.dense(h, x_dim)
    bn.deterministic("x_mean", tf.sigmoid(x_logits))
    bn.bernoulli("x", x_logits, group_ndims=1)
    return bn

Define the variational posterior model.
- TODO: complete the network

In [107]:
@zs.reuse_variables(scope="q_net")
def build_q_net(x, z_dim, n_z_per_x):
    '''
        TODO: define a Bayesian network
        HINT: see the generative model
    '''
    bn = zs.BayesianNet()
    
    x = tf.cast(x, tf.float32)
    '''
        TODO: add two more mlp layers of 500 hidden units
        HINT: from x to h
    '''
    h = tf.layers.dense(x, 500, activation=tf.nn.relu)
    h = tf.layers.dense(h, 500, activation=tf.nn.relu)
    
    z_mean = tf.layers.dense(h, z_dim)
    z_logstd = tf.layers.dense(h, z_dim)
    '''
        TODO: define q(z|x) using the Gaussian distribution of ZhuSuan
            > given input
                - "z", z_mean, z_logstd (note that it is not std), n_z_per_x
                - set group_ndims as 1
            > e.g.
                - z = bn.normal("z", ..., logstd=..., group_ndims=..., n_samples=...)
    '''
    bn.normal("z", z_mean, logstd=z_logstd, group_ndims=1, n_samples=n_z_per_x)
    
    return bn

## IV. Set the Hyperparameters

The following hyperparameters work well. However, you can modify them anyway.

In [108]:
x_dim = 28 * 28
z_dim = 40
n_particles = 1

learning_rate = 0.001
epochs = 3000
batch_size = 128
save_freq = 10

## V. Build the Model and the Loss

In [109]:
x_input = tf.placeholder(tf.float32, shape=[None, x_dim], name="x")
# Change x to binary form.
x = tf.cast(tf.less(tf.random_uniform(tf.shape(x_input)), x_input), tf.int32)
n = tf.placeholder(tf.int32, shape=[], name="n")

model = build_gen(x_dim, z_dim, n, n_particles)
variational = build_q_net(x, z_dim, n_particles)

# Define the ELBO and optimize it by SGVB.
lower_bound = zs.variational.elbo(model, {"x": x}, variational=variational, axis=0)
cost = tf.reduce_mean(lower_bound.sgvb())
lower_bound = tf.reduce_mean(lower_bound)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
infer_op = optimizer.minimize(cost)

x_gen = tf.reshape(model.observe()["x_mean"], [-1, 28, 28, 1])

## VI. Train the Model and Save the Generated Samples

In [110]:
# All generated samples are saved to this path.
result_path = 'images'
if not os.path.exists(result_path):
    os.mkdir(result_path)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(1, epochs + 1):
        time_epoch = -time.time()
        lbs = []
        for i_step, (x_batch, _) in enumerate(iterbatches(
            [data_train, label_train], batch_size=batch_size, include_final_partial_batch=False,
        )):
            x_batch = x_batch.reshape([-1, 28*28])
            _, lb = sess.run([infer_op, lower_bound],
                             feed_dict={x_input: x_batch,
                                        n: batch_size})
            lbs.append(lb)
        time_epoch += time.time()
        print("Epoch {} ({:.1f}s): Lower bound = {}".format(
            epoch, time_epoch, np.mean(lbs)))

        if epoch % save_freq == 0 or epoch == 1:
            images = sess.run(x_gen, feed_dict={n: 100})
            name = os.path.join(result_path,
                                "vae_epoch_{}.png".format(epoch))
            save_image_collections(images, name)

Epoch 1 (2.2s): Lower bound = -174.09962463378906


/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Epoch 2 (1.7s): Lower bound = -113.76283264160156
Epoch 3 (1.8s): Lower bound = -100.10377502441406
Epoch 4 (1.8s): Lower bound = -94.2618179321289
Epoch 5 (1.8s): Lower bound = -91.0434341430664
Epoch 6 (1.7s): Lower bound = -88.77664184570312
Epoch 7 (1.7s): Lower bound = -86.935791015625
Epoch 8 (1.7s): Lower bound = -85.39450073242188
Epoch 9 (1.7s): Lower bound = -84.27932739257812
Epoch 10 (1.7s): Lower bound = -83.26056671142578
Epoch 11 (1.7s): Lower bound = -82.2942886352539
Epoch 12 (1.7s): Lower bound = -81.52330017089844
Epoch 13 (1.7s): Lower bound = -80.92257690429688
Epoch 14 (1.7s): Lower bound = -80.37549591064453
Epoch 15 (1.7s): Lower bound = -79.82007598876953
Epoch 16 (1.7s): Lower bound = -79.39813232421875
Epoch 17 (1.7s): Lower bound = -79.02234649658203
Epoch 18 (1.7s): Lower bound = -78.6000747680664
Epoch 19 (1.7s): Lower bound = -78.40898895263672
Epoch 20 (1.7s): Lower bound = -78.00037384033203
Epoch 21 (1.7s): Lower bound = -77.69020080566406
Epoch 22 (1.

Epoch 166 (1.7s): Lower bound = -69.42371368408203
Epoch 167 (1.7s): Lower bound = -69.41143035888672
Epoch 168 (1.7s): Lower bound = -69.39197540283203
Epoch 169 (1.7s): Lower bound = -69.345947265625
Epoch 170 (1.7s): Lower bound = -69.344970703125
Epoch 171 (1.8s): Lower bound = -69.39170837402344
Epoch 172 (1.8s): Lower bound = -69.37326049804688
Epoch 173 (1.7s): Lower bound = -69.36666107177734
Epoch 174 (1.7s): Lower bound = -69.32002258300781
Epoch 175 (1.7s): Lower bound = -69.30078887939453
Epoch 176 (1.7s): Lower bound = -69.2745361328125
Epoch 177 (1.7s): Lower bound = -69.29397583007812
Epoch 178 (1.8s): Lower bound = -69.16412353515625
Epoch 179 (1.7s): Lower bound = -69.30355072021484
Epoch 180 (1.7s): Lower bound = -69.25865936279297
Epoch 181 (1.7s): Lower bound = -69.15545654296875
Epoch 182 (1.7s): Lower bound = -69.19557189941406
Epoch 183 (1.7s): Lower bound = -69.15924072265625
Epoch 184 (1.7s): Lower bound = -69.16091918945312
Epoch 185 (1.7s): Lower bound = -69.

Epoch 328 (1.7s): Lower bound = -67.9020004272461
Epoch 329 (1.7s): Lower bound = -67.9004898071289
Epoch 330 (1.7s): Lower bound = -67.87425231933594
Epoch 331 (1.7s): Lower bound = -68.00020599365234
Epoch 332 (1.7s): Lower bound = -67.84147644042969
Epoch 333 (1.7s): Lower bound = -67.876708984375
Epoch 334 (1.7s): Lower bound = -67.93082427978516
Epoch 335 (1.8s): Lower bound = -67.85472869873047
Epoch 336 (1.7s): Lower bound = -67.84779357910156
Epoch 337 (1.7s): Lower bound = -67.89330291748047
Epoch 338 (1.7s): Lower bound = -67.89917755126953
Epoch 339 (1.7s): Lower bound = -67.86872863769531
Epoch 340 (1.7s): Lower bound = -67.84419250488281
Epoch 341 (1.7s): Lower bound = -67.85530090332031
Epoch 342 (1.7s): Lower bound = -67.8399429321289
Epoch 343 (1.7s): Lower bound = -67.84900665283203
Epoch 344 (1.7s): Lower bound = -67.83070373535156
Epoch 345 (1.7s): Lower bound = -67.80915069580078
Epoch 346 (1.7s): Lower bound = -67.84335327148438
Epoch 347 (1.7s): Lower bound = -67.

...

Epoch 2248 (1.8s): Lower bound = -65.6182861328125
Epoch 2249 (1.7s): Lower bound = -65.56710815429688
Epoch 2250 (1.7s): Lower bound = -65.62014770507812
Epoch 2251 (1.7s): Lower bound = -65.54207611083984
Epoch 2252 (1.7s): Lower bound = -65.49215698242188
Epoch 2253 (1.8s): Lower bound = -65.52207946777344
Epoch 2254 (1.8s): Lower bound = -65.56632232666016
Epoch 2255 (1.8s): Lower bound = -65.49282836914062
Epoch 2256 (1.8s): Lower bound = -65.57963562011719
Epoch 2257 (1.7s): Lower bound = -65.6483383178711
Epoch 2258 (1.7s): Lower bound = -65.51422882080078
Epoch 2259 (1.8s): Lower bound = -65.50959777832031
Epoch 2260 (1.8s): Lower bound = -65.52974700927734
Epoch 2261 (1.7s): Lower bound = -65.50467681884766
Epoch 2262 (1.7s): Lower bound = -65.57667541503906
Epoch 2263 (1.8s): Lower bound = -65.5551986694336
Epoch 2264 (1.8s): Lower bound = -65.64907836914062
Epoch 2265 (2.0s): Lower bound = -65.59053802490234
Epoch 2266 (1.8s): Lower bound = -65.54888916015625
Epoch 2267 (1.7

Epoch 2407 (1.7s): Lower bound = -65.5985336303711
Epoch 2408 (1.7s): Lower bound = -65.50765991210938
Epoch 2409 (1.7s): Lower bound = -65.42865753173828
Epoch 2410 (1.7s): Lower bound = -65.48192596435547
Epoch 2411 (1.7s): Lower bound = -65.48991394042969
Epoch 2412 (1.7s): Lower bound = -65.50643157958984
Epoch 2413 (1.7s): Lower bound = -65.57654571533203
Epoch 2414 (1.7s): Lower bound = -65.49073791503906
Epoch 2415 (1.7s): Lower bound = -65.49376678466797
Epoch 2416 (1.7s): Lower bound = -65.51013946533203
Epoch 2417 (1.7s): Lower bound = -65.48699951171875
Epoch 2418 (1.8s): Lower bound = -65.47339630126953
Epoch 2419 (1.7s): Lower bound = -65.5457534790039
Epoch 2420 (1.7s): Lower bound = -65.46917724609375
Epoch 2421 (1.7s): Lower bound = -65.52156066894531
Epoch 2422 (1.7s): Lower bound = -65.46429443359375
Epoch 2423 (1.7s): Lower bound = -65.5451889038086
Epoch 2424 (1.7s): Lower bound = -65.48500061035156
Epoch 2425 (1.7s): Lower bound = -65.57413482666016
Epoch 2426 (1.7

Epoch 2566 (1.7s): Lower bound = -65.4415283203125
Epoch 2567 (1.7s): Lower bound = -65.46824645996094
Epoch 2568 (1.8s): Lower bound = -65.4579849243164
Epoch 2569 (1.7s): Lower bound = -65.47064208984375
Epoch 2570 (1.8s): Lower bound = -65.4467544555664
Epoch 2571 (1.7s): Lower bound = -65.46589660644531
Epoch 2572 (1.7s): Lower bound = -65.48060607910156
Epoch 2573 (1.7s): Lower bound = -65.45562744140625
Epoch 2574 (1.8s): Lower bound = -65.45706939697266
Epoch 2575 (1.9s): Lower bound = -65.48789978027344
Epoch 2576 (1.8s): Lower bound = -65.49150085449219
Epoch 2577 (1.7s): Lower bound = -65.48983001708984
Epoch 2578 (1.7s): Lower bound = -65.43085479736328
Epoch 2579 (1.8s): Lower bound = -65.40453338623047
Epoch 2580 (1.8s): Lower bound = -65.48688507080078
Epoch 2581 (1.7s): Lower bound = -65.43293762207031
Epoch 2582 (1.7s): Lower bound = -65.4537353515625
Epoch 2583 (1.7s): Lower bound = -65.42517852783203
Epoch 2584 (1.7s): Lower bound = -65.4964370727539
Epoch 2585 (1.7s)

Epoch 2725 (1.8s): Lower bound = -65.41796112060547
Epoch 2726 (1.7s): Lower bound = -65.42849731445312
Epoch 2727 (1.7s): Lower bound = -65.49919128417969
Epoch 2728 (1.8s): Lower bound = -65.3665542602539
Epoch 2729 (1.8s): Lower bound = -65.43315124511719
Epoch 2730 (1.7s): Lower bound = -65.40174865722656
Epoch 2731 (1.7s): Lower bound = -65.44268035888672
Epoch 2732 (1.8s): Lower bound = -65.4053726196289
Epoch 2733 (1.7s): Lower bound = -65.45626831054688
Epoch 2734 (1.8s): Lower bound = -65.3740234375
Epoch 2735 (1.7s): Lower bound = -65.31121826171875
Epoch 2736 (1.7s): Lower bound = -65.4367446899414
Epoch 2737 (1.7s): Lower bound = -65.44383239746094
Epoch 2738 (1.7s): Lower bound = -65.36941528320312
Epoch 2739 (1.7s): Lower bound = -65.48538970947266
Epoch 2740 (1.8s): Lower bound = -65.37178802490234
Epoch 2741 (1.8s): Lower bound = -65.35965728759766
Epoch 2742 (1.7s): Lower bound = -65.45086669921875
Epoch 2743 (1.7s): Lower bound = -65.40252685546875
Epoch 2744 (1.7s): 

Epoch 2884 (1.8s): Lower bound = -65.4017562866211
Epoch 2885 (1.7s): Lower bound = -65.38760375976562
Epoch 2886 (1.7s): Lower bound = -65.35747528076172
Epoch 2887 (1.7s): Lower bound = -65.38440704345703
Epoch 2888 (1.7s): Lower bound = -65.35105895996094
Epoch 2889 (1.7s): Lower bound = -65.396240234375
Epoch 2890 (1.7s): Lower bound = -65.33109283447266
Epoch 2891 (1.7s): Lower bound = -65.37974548339844
Epoch 2892 (1.7s): Lower bound = -65.38993072509766
Epoch 2893 (1.7s): Lower bound = -65.40658569335938
Epoch 2894 (1.7s): Lower bound = -65.32606506347656
Epoch 2895 (1.7s): Lower bound = -65.37419128417969
Epoch 2896 (1.7s): Lower bound = -65.40142822265625
Epoch 2897 (1.7s): Lower bound = -65.38774108886719
Epoch 2898 (1.7s): Lower bound = -65.40360260009766
Epoch 2899 (1.7s): Lower bound = -65.35384368896484
Epoch 2900 (1.8s): Lower bound = -65.3573989868164
Epoch 2901 (1.7s): Lower bound = -65.46168518066406
Epoch 2902 (1.7s): Lower bound = -65.33549499511719
Epoch 2903 (1.7s